# Prompt Chaining against GOD Tasks¨
Um LLMs nicht zu überfordern hilft es grössere Aufgaben in Teil-Schritte zu zerlegen und einzelne LLMs aneinanderzukette. Durch die einfacheren Aufgaben können wir auf kleinere Modelle zurückgreifen, die viel kosteneffektiver sind.

In [2]:
# prompt_chaining_support_ticket.py

import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# --- 1. Konfiguration und Initialisierung ---

# Lade alle Umgebungsvariablen aus der .env Datei
load_dotenv()

# Konfiguration für Azure OpenAI
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

if not api_key or not azure_endpoint or not azure_deployment:
    raise ValueError("Azure OpenAI Konfiguration fehlt. Bitte überprüfe deine .env Datei.")

# Initialisiere den Azure OpenAI Client
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint,
)

# Beispiel Support Tickets beantworten
Um ein Support Ticket abzuarbeiten sind oft mehrere Schritte nötig und Informationen müssen herangeschafft werden

In [4]:
# Statische Variablen für dieses Beispiel
customer_data = {
        "Kundenname": "Herr Müller",
        "Anrede": "Sehr geehrter Herr Müller",
        "Unternehmen": "Müller AG",
        "Ticket-ID": "TICK-2025-0915"
    }
    
support_ticket = """
    Betreff: Drucker druckt nur noch leere Seiten
    
    Hallo Support-Team,
    
    unser Bürodrucker (Modell: HP OfficeJet Pro 9010) im zweiten Stock hat seit heute Morgen ein Problem.
    Egal was wir versuchen zu drucken, es kommen nur noch leere Seiten heraus.
    Wir haben bereits die Tintenpatronen überprüft, sie sind alle voll. Ein Neustart des Druckers und der verbundenen Computer hat auch nichts gebracht.
    
    Können Sie uns bitte helfen?
    
    Danke und Gruss
    """

# Die Lösung
Wir zerteilen die Aufgabe in 3 verschiedene "Agents" einen Planer einer Worker und einen Support Agent

In [5]:
# Planer Agent

def agent_planner(ticket_content: str) -> str:
    """Agent 1: Erstellt einen 5-stufigen Handlungsplan."""
    print("--- Agent 1 (Planner) wird ausgeführt... ---")
    
    system_prompt = """Du bist ein Senior IT-Support-Stratege. 
    Deine einzige Aufgabe ist es, für das folgende Support-Ticket einen klaren und prägnanten 5-stufigen Handlungsplan zu erstellen. 
    Löse das Problem nicht selbst, erstelle nur den Plan."""
    
    response = client.chat.completions.create(
        model=azure_deployment,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": ticket_content}
        ],
        temperature=0.0
    )
    return response.choices[0].message.content

plan = agent_planner(support_ticket)
print(f"\n** Generierter Handlungsplan:**\n{plan}\n")
    

--- Agent 1 (Planner) wird ausgeführt... ---

** Generierter Handlungsplan:**
Handlungsplan zur Behebung des Problems "Drucker druckt nur noch leere Seiten":

1. **Druckkopfreinigung durchführen**  
   Führen Sie eine automatische Druckkopfreinigung über das Druckermenü oder die Druckersoftware auf dem Computer durch, um verstopfte Düsen zu beseitigen.

2. **Druckertreiber überprüfen und neu installieren**  
   Prüfen Sie, ob der Druckertreiber aktuell ist. Deinstallieren Sie den vorhandenen Treiber und installieren Sie die neueste Version von der HP-Website.

3. **Testseite direkt vom Drucker drucken**  
   Drucken Sie eine Testseite direkt über das Druckermenü, um zu überprüfen, ob das Problem hardwareseitig besteht.

4. **Tintenpatronen erneut einsetzen und Kontakte reinigen**  
   Nehmen Sie die Tintenpatronen heraus, reinigen Sie die Kontakte vorsichtig mit einem fusselfreien Tuch und setzen Sie sie wieder korrekt ein.

5. **Firmware-Update prüfen und durchführen**  
   Überprüfen

# Problem Löser 
Nun kriegt der Problem-Löser das Support-Ticket plus den Lösungsansatz vom Planer als Input

In [6]:
def agent_solver(ticket_content: str, action_plan: str) -> str:
    """Agent 2: Löst das Problem anhand des vorgegebenen Plans."""
    print("--- Agent 2 (Solver) wird ausgeführt... ---")
    system_prompt = "Du bist ein technischer IT-Spezialist. " \
    "Du erhältst ein Support-Ticket und einen Handlungsplan. " \
    "Deine Aufgabe ist es, den Plan Schritt für Schritt 'abzuarbeiten' und eine detaillierte technische Lösung zu verfassen, die beschreibt, was getan wurde."
    
    combined_input = f"**Support Ticket:**\n{ticket_content}\n\n**Handlungsplan:**\n{action_plan}"
    
    response = client.chat.completions.create(
        model=azure_deployment,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": combined_input}
        ],
        temperature=0.3 # Etwas mehr Kreativität für die Lösungsbeschreibung
    )
    return response.choices[0].message.content

solution = agent_solver(support_ticket, plan)
print(f"\n** Generierte technische Lösung:**\n{solution}\n")

--- Agent 2 (Solver) wird ausgeführt... ---

** Generierte technische Lösung:**
**Technische Lösung zur Behebung des Problems "Drucker druckt nur noch leere Seiten" beim HP OfficeJet Pro 9010**

---

**1. Druckkopfreinigung durchführen**

- Zugriff auf das Druckermenü am Gerät:
  - Navigieren Sie im Drucker-Display zu „Setup“ > „Wartung“ > „Druckkopfreinigung“.
  - Starten Sie den automatischen Reinigungsprozess.
- Alternativ über die Druckersoftware auf dem Computer:
  - Öffnen Sie die HP Smart App oder die HP Druckersoftware.
  - Wählen Sie den Drucker aus und starten Sie die Funktion „Druckkopfreinigung“.
- Nach Abschluss der Reinigung drucken Sie eine Testseite, um zu prüfen, ob die Ausgabe verbessert wurde.

**Ergebnis:** Die automatische Druckkopfreinigung beseitigt verstopfte Düsen, die häufig Ursache für leere Seiten sind.

---

**2. Druckertreiber überprüfen und neu installieren**

- Überprüfen Sie die aktuell installierte Treiberversion:
  - Öffnen Sie auf dem PC die Systemst

# Support Agent
Nachdem nun das Problem intern bearbeitet wurde, erzeugt ein dritter "Agent" die Antwort-E-Mail an den Ticket-Steller. Hier würde nun typischerweise ein HITL Ansatz gewählt werden. CWR -> über längere Sicht

In [9]:
def agent_notifier(ticket_content: str, solution_details: str, customer: dict) -> str:
    """Agent 3: Formuliert eine kundenfreundliche E-Mail."""
    print("--- Agent 3 (Notifier) wird ausgeführt... ---")
    system_prompt = """Du bist ein Experte für Kundenkommunikation im IT-Support.
    Du erhältst ein ursprüngliches Ticket, die technische Lösung, die von uns durchgeführt wurde und Kundendaten. 
    Deine Aufgabe ist es, eine höfliche, professionelle und leicht verständliche E-Mail an den Kunden zu schreiben. 
    Informiere ihn, dass sein Problem gelöst wurde, und fasse die Lösung kurz zusammen.
    Unterzeichne deine E-Mail mit Susi-Support von der Susi-Support AG."""
    
    customer_info = "\n".join([f"{key}: {value}" for key, value in customer.items()])
    combined_input = f"**Kundendaten für die E-Mail:**\n{customer_info}\n\n**Ursprüngliches Ticket:**\n{ticket_content}\n\n**Technische Lösung:**\n{solution_details}"

    response = client.chat.completions.create(
        model=azure_deployment,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": combined_input}
        ],
        temperature=0.5 # Mehr Kreativität für eine natürlich klingende E-Mail
    )
    return response.choices[0].message.content

final_email = agent_notifier(support_ticket, solution, customer_data)
print(f"\n**Finale Kunden-E-Mail:**\n{final_email}\n")


--- Agent 3 (Notifier) wird ausgeführt... ---

**Finale Kunden-E-Mail:**
Betreff: Ihr Ticket TICK-2025-0915 – Problem mit Ihrem HP OfficeJet Pro 9010 behoben

Sehr geehrter Herr Müller,

wir freuen uns, Ihnen mitteilen zu können, dass das Problem mit Ihrem HP OfficeJet Pro 9010, bei dem nur noch leere Seiten gedruckt wurden, erfolgreich behoben wurde.

Zur Lösung haben wir folgende Schritte durchgeführt:
- Eine automatische Druckkopfreinigung, um verstopfte Düsen zu beseitigen.
- Überprüfung und Neuinstallation des aktuellen Druckertreibers.
- Reinigung der Kontakte der Tintenpatronen und korrektes Wiedereinsetzen.
- Prüfung und Installation eines verfügbaren Firmware-Updates.
- Testdruck direkt vom Drucker zur Sicherstellung der Funktionalität.

Nach diesen Maßnahmen sollte Ihr Drucker nun wieder einwandfrei drucken. Sollten Sie weiterhin Probleme feststellen, stehen wir Ihnen selbstverständlich gern zur Verfügung.

Vielen Dank für Ihr Vertrauen in die Müller AG.

Freundliche Grüße  


In [ ]:


def agent_notifier(ticket_content: str, solution_details: str, customer: dict) -> str:
    """Agent 3: Formuliert eine kundenfreundliche E-Mail."""
    print("--- ✉️ Agent 3 (Notifier) wird ausgeführt... ---")
    system_prompt = "Du bist ein Experte für Kundenkommunikation im IT-Support. Du erhältst ein ursprüngliches Ticket, die technische Lösung und Kundendaten. Deine Aufgabe ist es, eine höfliche, professionelle und leicht verständliche E-Mail an den Kunden zu schreiben. Informiere ihn, dass sein Problem gelöst wurde, und fasse die Lösung kurz zusammen."
    
    customer_info = "\n".join([f"{key}: {value}" for key, value in customer.items()])
    combined_input = f"**Kundendaten für die E-Mail:**\n{customer_info}\n\n**Ursprüngliches Ticket:**\n{ticket_content}\n\n**Technische Lösung:**\n{solution_details}"

    response = client.chat.completions.create(
        model=azure_deployment,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": combined_input}
        ],
        temperature=0.5 # Mehr Kreativität für eine natürlich klingende E-Mail
    )
    return response.choices[0].message.content


# --- 3. Die Hauptfunktion, die die Kette orchestriert ---

def main():
    """Definiert die Inputs und führt die Agenten-Kette aus."""
    
    # Statische Variablen für dieses Beispiel
    customer_data = {
        "Kundenname": "Herr Müller",
        "Anrede": "Sehr geehrter Herr Müller",
        "Unternehmen": "Müller AG",
        "Ticket-ID": "TICK-2025-0915"
    }
    
    support_ticket = """
    Betreff: Drucker druckt nur noch leere Seiten
    
    Hallo Support-Team,
    
    unser Bürodrucker (Modell: HP OfficeJet Pro 9010) im zweiten Stock hat seit heute Morgen ein Problem.
    Egal was wir versuchen zu drucken, es kommen nur noch leere Seiten heraus.
    Wir haben bereits die Tintenpatronen überprüft, sie sind alle voll. Ein Neustart des Druckers und der verbundenen Computer hat auch nichts gebracht.
    
    Können Sie uns bitte helfen?
    
    Danke und Gruss
    """
    
    print("==============================================")
    print("🚀 Starte Prompt-Chaining-Prozess...")
    print("==============================================\n")
    print(f"**Eingehendes Ticket:**\n{support_ticket}\n")

    # Kette ausführen: Output von Agent 1 wird Input für Agent 2, etc.
    plan = agent_planner(support_ticket)
    print(f"\n**✅ Generierter Handlungsplan:**\n{plan}\n")
    
    solution = agent_solver(support_ticket, plan)
    print(f"\n**✅ Generierte technische Lösung:**\n{solution}\n")
    
    final_email = agent_notifier(support_ticket, solution, customer_data)
    print(f"\n**✅ Finale Kunden-E-Mail:**\n{final_email}\n")
    
    print("==============================================")
    print("🏁 Prozess abgeschlossen.")
    print("==============================================")


if __name__ == "__main__":
    main()

🚀 Starte Prompt-Chaining-Prozess...

**Eingehendes Ticket:**

    Betreff: Drucker druckt nur noch leere Seiten
    
    Hallo Support-Team,
    
    unser Bürodrucker (Modell: HP OfficeJet Pro 9010) im zweiten Stock hat seit heute Morgen ein Problem.
    Egal was wir versuchen zu drucken, es kommen nur noch leere Seiten heraus.
    Wir haben bereits die Tintenpatronen überprüft, sie sind alle voll. Ein Neustart des Druckers und der verbundenen Computer hat auch nichts gebracht.
    
    Können Sie uns bitte helfen?
    
    Danke und Gruss
    

--- 🧠 Agent 1 (Planner) wird ausgeführt... ---

**✅ Generierter Handlungsplan:**
Handlungsplan zur Behebung des Problems "Drucker druckt nur noch leere Seiten":

1. **Druckkopfreinigung durchführen**  
   Anleitung zur automatischen oder manuellen Reinigung des Druckkopfs am HP OfficeJet Pro 9010 bereitstellen und durchführen, um verstopfte Düsen auszuschließen.

2. **Druckertreiber und Verbindung prüfen**  
   Sicherstellen, dass der Druckert